In [ ]:
#default_exp training

In [ ]:
#export
import numpy as np
import pandas as pd
import os
from deeplearning_image_classification import data_loading

from sklearn import model_selection, metrics
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
from tensorflow import keras
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras import applications
#export


np.random.seed(0)
pd.set_option('display.max_colwidth', 100)
DATA_DIR = data_loading.DATA_DIR

In [ ]:
%cd ..

/home/kuba/Projects/deeplearning_image_classification


Try removing this line on your PC, maybe you will not encounter [this issue](https://github.com/tensorflow/tensorflow/issues/24496)

In [ ]:
    
#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)

In [ ]:
#export
train_csv_path = os.path.join(DATA_DIR, 'train_metadata.csv')
test_csv_path = os.path.join(DATA_DIR, 'test_metadata.csv')

In [ ]:
#export
def get_metadata(train_metadata_path=train_csv_path, test_metadata_path=test_csv_path):
    train_metadata_df = pd.read_csv(train_csv_path)
    test_metadata_df = pd.read_csv(test_csv_path)
    return train_metadata_df, test_metadata_df

In [ ]:
train_csv_path

'data/train_metadata.csv'

In [ ]:
train_metadata_df, test_metadata_df = get_metadata()

In [ ]:
sample_size = 11000
sample_val_size = 1000
__, sample_train_val_metadata_df = model_selection.train_test_split(train_metadata_df, test_size=sample_size, random_state=2, stratify=train_metadata_df['class'])
sample_train_metadata_df, sample_val_metadata_df = model_selection.train_test_split(sample_train_val_metadata_df, test_size=sample_val_size, random_state=2, stratify=sample_train_val_metadata_df['class'])

In [ ]:
#export
image_size = (224, 224)
train_batch_size = 32
val_batch_size = 32
test_batch_size = 64


def get_train_val_test_iterators(train_metadata_df, val_metadata_df, test_metadata_df, image_size, train_batch_size, val_batch_size, test_batch_size):
    image_gen = keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=keras.applications.mobilenet_v2.preprocess_input,
        horizontal_flip=True
    )

    train_image_iterator = image_gen.flow_from_dataframe(train_metadata_df, batch_size=train_batch_size, target_size=image_size,
        shuffle=False)
    val_image_iterator = image_gen.flow_from_dataframe(val_metadata_df, batch_size=val_batch_size, target_size=image_size, shuffle=False)
    test_image_iterator = image_gen.flow_from_dataframe(test_metadata_df, batch_size=test_batch_size, target_size=image_size, shuffle=False)
    return train_image_iterator, val_image_iterator, test_image_iterator

In [ ]:
train_image_iterator, val_image_iterator, test_image_iterator = get_train_val_test_iterators(sample_train_metadata_df, sample_val_metadata_df, test_metadata_df, image_size=image_size, train_batch_size=train_batch_size, test_batch_size=test_batch_size, val_batch_size=val_batch_size) 
n_classes = len(train_image_iterator.class_indices)

Found 10000 validated image filenames belonging to 82 classes.
Found 1000 validated image filenames belonging to 82 classes.
Found 10000 validated image filenames belonging to 82 classes.


# Setting up model

We use pretrained MobileNet model for feature extraction

In [ ]:
#export
learning_rate = 0.001
beta_1 = 0.9
beta_2 = 0.999
optimizer = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2)
default_metrics = ['acc', keras.metrics.Precision(), keras.metrics.Recall()]


def setup_model(n_classes, optimizer, metrics):
    base_model = keras.applications.MobileNet(include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    model = keras.Sequential(
        [
            base_model,
            keras.layers.AveragePooling2D((2, 2)),
            keras.layers.Convolution2D(64, (2, 2)),
            keras.layers.Flatten(),
            keras.layers.Dense(n_classes),
            keras.layers.Softmax()
        ])
    model.compile(
        loss=keras.losses.CategoricalCrossentropy(),
        optimizer=optimizer,
        metrics=metrics)
    return model

In [ ]:
model = setup_model(n_classes, optimizer, default_metrics)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 7, 7, 1024)        3228864   
_________________________________________________________________
average_pooling2d (AveragePo (None, 3, 3, 1024)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 2, 2, 64)          262208    
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 82)                21074     
_________________________________________________________________
softmax (Softmax)            (None, 82)                0         
Total params: 3,512,146
Trainable params: 283,282
Non-trainable params: 3,228,864
________________________________________

In [ ]:
model.fit(train_image_iterator, validation_data=val_image_iterator, epochs=10)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 313 steps, validate for 32 steps
Epoch 1/10
 94/313 [========>.....................] - ETA: 46s - loss: 6.3973 - acc: 0.1016 - precision: 0.1499 - recall: 0.0638

In [ ]:
y_test_pred = model.predict(test_image_iterator).argmax(axis=1)

In [ ]:
y_test = np.array([test_image_iterator.class_indices[c] for c in test_metadata_df['class']])

## Test set accuracy

In [ ]:
metrics.accuracy_score(y_test, y_test_pred)

## Script mode

In [ ]:
#export
sample_size = 11000
sample_val_size = 1000
epochs = 10


if __name__ == '__main__':
    print(DATA_DIR)
    train_metadata_df, test_metadata_df = get_metadata()
    __, sample_train_val_metadata_df = model_selection.train_test_split(train_metadata_df, test_size=sample_size, random_state=2, stratify=train_metadata_df['class'])
    sample_train_metadata_df, sample_val_metadata_df = model_selection.train_test_split(sample_train_val_metadata_df, test_size=sample_val_size, random_state=2, stratify=sample_train_val_metadata_df['class'])

    train_image_iterator, val_image_iterator, test_image_iterator = get_train_val_test_iterators(
        sample_train_metadata_df, sample_val_metadata_df, test_metadata_df,
        image_size=image_size, train_batch_size=train_batch_size, test_batch_size=test_batch_size, val_batch_size=val_batch_size) 
    n_classes = len(train_image_iterator.class_indices)
    model = setup_model(n_classes, optimizer, default_metrics)
    model.fit(train_image_iterator, validation_data=val_image_iterator, epochs=10)